### Selecting columns, viualizing

In [1]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
import itertools

In [2]:
data = pd.read_csv("../dat/data_clean.csv")

/home/daniel/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (5,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273543 entries, 0 to 273542
Data columns (total 20 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   tconst                       273543 non-null  object 
 1   titleType                    273543 non-null  object 
 2   primaryTitle                 273543 non-null  object 
 3   originalTitle                273543 non-null  object 
 4   isAdult                      273543 non-null  int64  
 5   startYear                    273543 non-null  object 
 6   endYear                      273543 non-null  object 
 7   runtimeMinutes               273543 non-null  object 
 8   genres                       273543 non-null  object 
 9   averageRating                273543 non-null  float64
 10  numVotes                     273543 non-null  int64  
 11  Budget                       49881 non-null   float64
 12  Gross US & Canada            18982 non-null   float64
 13 

In [4]:
data["Critic reviews"] = data["Critic reviews"].fillna(0)
data["User reviews"] = data["User reviews"].fillna(0)

data["isAdult2"] = data.apply(lambda row: int("Adult" in row["genres"]), axis=1)
print(len(data[data["isAdult"] != data["isAdult2"]]))  # We use the one based on the genres

# tconst was only required for joins
# titleType is only films for us, we filtered them
# we do not use the titles as predictors
# endYear is None for all films
# isAdult will be added back in a consistent format later on
data = data.drop(columns=["tconst", "titleType", "primaryTitle", "originalTitle", "endYear", "isAdult", "isAdult2"])

318


In [5]:
genre_list = data["genres"].unique().tolist()
for i, entry in enumerate(genre_list):
    genre_list[i] = entry.split(",")

genre_set = set(itertools.chain(*genre_list))
print(genre_set)
# Here short is a weird category, as there is also a short category in the titleType column, where we only
# kept films. Here the IMDb database is a bit inconsistent.

{'Drama', 'Sci-Fi', 'Musical', 'Biography', 'History', 'Documentary', 'Family', 'Western', 'Crime', 'News', 'Fantasy', 'Reality-TV', 'Short', 'Sport', 'Animation', 'Film-Noir', 'War', 'Comedy', 'Thriller', '\\N', 'Adventure', 'Mystery', 'Horror', 'Music', 'Action', 'Romance', 'Adult', 'Talk-Show', 'Game-Show'}


In [6]:
for genre in genre_set:
    print(genre, end=" ")
    data[f"is{genre}"] = data.apply(lambda row: int(genre in row["genres"]), axis=1)

Drama Sci-Fi Musical Biography History Documentary Family Western Crime News Fantasy Reality-TV Short Sport Animation Film-Noir War Comedy Thriller \N Adventure Mystery Horror Music Action Romance Adult Talk-Show Game-Show 

In [7]:
for genre in genre_set:
    print(genre, data[f"is{genre}"].sum())

Drama 121169
Sci-Fi 6498
Musical 6177
Biography 8245
History 7525
Documentary 40450
Family 10613
Western 5034
Crime 23356
News 652
Fantasy 8091
Reality-TV 43
Short 24
Sport 3236
Animation 4812
Film-Noir 761
War 5882
Comedy 67803
Thriller 22321
\N 10234
Adventure 17205
Mystery 10196
Horror 19359
Music 12423
Action 28352
Romance 30430
Adult 4229
Talk-Show 8
Game-Show 2


In [8]:
# We drop categories that do not have enough samples to be representative
# We draw the line at 100
# We also drop writers and directors. These are interesting features,
# but having them as binary columns would be infeasible.
# Genres are added as binary predictors, thus the genres column is no longer used.
data = data.drop(columns=["isShort", "isReality-TV", "is\\N", "isTalk-Show", "directors", "writers", "genres"])

In [ ]:
for rating in data.Rating.unique():
    print(rating, end=" ")
    data[f"is{rating}"] = data.apply(lambda row: int(row["Rating"] == rating), axis=1)

nan Not Rated Unrated TV-PG Passed Approved G TV-14 TV-G PG-13 PG TV-MA Open GP R M/PG TV-Y7 M X TV-13 TV-Y7-FV NC-17 AO E (Banned) 12 TV-Y T MA-13 18 E10+ MA-17 

In [ ]:
for rating in data.Rating.unique():
    print(rating, data[f"is{rating}"].sum())
    
# Not needed anymore, we have the binary version
data = data.drop(columns=["Rating"])

In [ ]:
# Again, we drop ratings that do not have enough samples to be representative
# We draw the line at 100
data = data.drop(columns=[
    "isnan", "isOpen", "isM/PG", "isTV-Y7", "isM", "isTV-13",
    "isTV-Y7-FV", "isNC-17", "isAO", "is(Banned)", "isE",
    "is12", "isTV-Y", "is18", "isE10+", "isMA-17"]
)

In [ ]:
def clean_unknowns(row, column):
    if row[column] == "\\N":
        return None
    else:
        return row[column]

def clean_reviews(row, column):
    if isinstance(row[column], str) and "K" in row[column]:
        # print(row[column], end=" -> ")
        if "." in row[column]:
            # print(int(row[column][:-3]) * 1000 + int(row[column][-2]) * 100)
            return int(row[column][:-3]) * 1000 + int(row[column][-2]) * 100
        else:
            # print(int(row[column][:-1]) * 1000)
            return int(row[column][:-1]) * 1000
    else:
        return row[column]

# Just an example of problematic data types
# print("Problematic form")
# print(data.startYear.unique())
# print(data.runtimeMinutes.unique())
# print(data["User reviews"].unique())
# print(data["Critic reviews"].unique())

data["startYear"] = data.apply(lambda row: clean_unknowns(row, "startYear"), axis=1)
data["runtimeMinutes"] = data.apply(lambda row: clean_unknowns(row, "runtimeMinutes"), axis=1)
data["User reviews"] = data.apply(lambda row: clean_reviews(row, "User reviews"), axis=1)
data["Critic reviews"] = data.apply(lambda row: clean_reviews(row, "Critic reviews"), axis=1)

for column in ["startYear", "runtimeMinutes", "User reviews", "Critic reviews"]:
    data[column] = pd.to_numeric(data[column])

# print("Resolved form")
# print(data.startYear.unique())
# print(data.runtimeMinutes.unique())
# print(data["User reviews"].unique())
# print(data["Critic reviews"].unique())

In [ ]:
print("All features present:")
filtered = data.dropna()
print(len(filtered))
print()

print("Opening weekend not present:")
filtered2 = data.drop(columns=["Opening weekend US & Canada"]).dropna()
print(len(filtered2))
print()

print("Opening weekend + Gross not present:")
filtered3 = data.drop(columns=["Opening weekend US & Canada", "Gross US & Canada"]).dropna()
print(len(filtered3))
print()

In [ ]:
filtered.corr()

In [ ]:
filtered3.isAdult.unique()